In [1]:
import sys
sys.path.insert(0, "../")

import numpy as np

import cobra.test
from ecoli.library import fba_gd


# Test case: OAA -> Asp -> Asn, Lys, Met, Thr

### Read full E. coli model from cobra, and extract a subset of reaction stoichiometries to work with

In [2]:
full_model = cobra.test.create_test_model("ecoli")  # iJO1366

stoichiometry = {}
for reaction in full_model.reactions:
    if reaction.id in [
        "ASPTA",  # -> Asp
        "ASNN",  # -> Asn

        "DHDPS",
        "DHDPRy",
        "THDPS",
        "SDPTA",
        "SDPDS",
        "DAPE",
        "DAPDC",  # -> Lus

        "HSST",
        "SHSL1",
        "CYSTL",
        "METS",  # -> Met

        "ASPK",
        "ASAD",
        "HSDy",
        "HSK",
        "THRS",  # -> Thr
    ]:
        stoichiometry[reaction.id] = {metabolite.id: coeff for metabolite, coeff in reaction.metabolites.items()}

exchanges=['5mthf_c', 'adp_c', 'akg_c', 'atp_c', 'co2_c', 'coa_c', 'cys__L_c', 'glu__L_c', 'h2o_c', 'h_c',
           'nadp_c', 'nadph_c', 'nh4_c', 'oaa_c', 'pi_c', 'pyr_c', 'succ_c', 'succoa_c', 'thf_c']
objective={"asn__L_c": 0.6, "asp__L_c": 1.0, "lys__L_c": 0.9, "met__L_c": 0.5, "thr__L_c": 2.}


### Set up the structure of the FBA problem

In [4]:
fba = fba_gd.GradientDescentFba(
    stoichiometry,
    exchanges=exchanges,
    objective=objective,
    objectiveType="homeostatic",
)

### Solve the problem, passing in current objectives

In [6]:
soln = fba.solve(objective, params=None)

# Steady-state residual is a measure of how far the solution is from steady-state
print(f"RMSE = {np.sqrt(np.sum(np.square(soln.ss_residual)))}")

RMSE = 1.2942808943932731e-07


### The solution object has solved values for reaction velocities, and rates of change (dm/dt) of all molecules

- Objective molecules have dm/dt close to their objective values
- Exchange molecules' dm/dt are as needed to support production of the objectives
- Internal (intermediate) molecules have dm/dt close to zero (i.e. steady state)

In [13]:
print("Fluxes")
for reaction_id in sorted(stoichiometry):
    print(f"  {reaction_id:>10}:   {soln.velocities[reaction_id]:+.2f}")

print()
print("Objectives")
for molecule in sorted(objective):
    print(f"  {molecule:>10}:   {soln.dm_dt[molecule]:.2f} / {objective[molecule]:.2f}")

print()
print("Exchanges")
for molecule in sorted(exchanges):
    print(f"  {molecule:>10}:   {soln.dm_dt[molecule]:+.2f}")

print()
print("Intermediates")
for molecule in fba.network.moleculeIDs():
    if molecule not in objective and molecule not in exchanges:
        print(f"  {molecule:>10}:   {soln.dm_dt[molecule]: .2e}")



Fluxes
        ASAD:   -3.40
        ASNN:   -0.60
        ASPK:   +3.40
       ASPTA:   -5.00
       CYSTL:   +0.50
       DAPDC:   +0.90
        DAPE:   +0.90
      DHDPRy:   +0.90
       DHDPS:   +0.90
        HSDy:   -2.50
         HSK:   +2.00
        HSST:   +0.50
        METS:   +0.50
       SDPDS:   +0.90
       SDPTA:   -0.90
       SHSL1:   +0.50
       THDPS:   +0.90
        THRS:   +2.00

Objectives
    asn__L_c:   0.60 / 0.60
    asp__L_c:   1.00 / 1.00
    lys__L_c:   0.90 / 0.90
    met__L_c:   0.50 / 0.50
    thr__L_c:   2.00 / 2.00

Exchanges
     5mthf_c:   -0.50
       adp_c:   +5.40
       akg_c:   +5.90
       atp_c:   -5.40
       co2_c:   +0.90
       coa_c:   +1.40
    cys__L_c:   -0.50
    glu__L_c:   -5.90
       h2o_c:   -1.90
         h_c:   -3.80
      nadp_c:   +6.80
     nadph_c:   -6.80
       nh4_c:   -0.10
       oaa_c:   -5.00
        pi_c:   +5.40
       pyr_c:   -0.40
      succ_c:   +1.40
    succoa_c:   -1.40
       thf_c:   +0.50

Intermediates
 